In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# 定义参数
region = 'daqing'
year = '2000'
startDate = year + '-01-01'
endDate = year + '-12-31'

In [4]:
# # 大庆市
roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

In [5]:
## 指数的计算
def water_index(img):
    image = img
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

# 数据集

In [ ]:
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(roi).filterDate('2015-04-01','2015-10-31').filter(ee.Filter.lt('CLOUD_COVER',10))
# print(l8.aggregate_array('system:id').getInfo())

# l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterBounds(roi).filterDate('2000-04-01','2000-10-31').filter(ee.Filter.lt('CLOUD_COVER',10))
# print(l5.aggregate_array('system:id').getInfo())

In [6]:
#  'LANDSAT/LT05/C01/T1_SR/LT05_119028_20000528', 
#  'LANDSAT/LT05/C01/T1_SR/LT05_119028_20000816',
  
#  'LANDSAT/LT05/C01/T1_SR/LT05_120028_20000706', 
#  'LANDSAT/LT05/C01/T1_SR/LT05_120028_20000924', 

a = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_119028_20000816').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
# a = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_120028_20200526')
image = maskSR(water_index(a))
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
Map.addLayer(image.clip(roi), visParams,'a image')
Map

Map(bottom=46743.0, center=[46.356595404115275, 124.62049999999992], controls=(WidgetControl(options=['positio…

In [ ]:
# # dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
# #             .filterDate('2020-05-25', '2020-05-28') \
# #             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
# #             .filterBounds(roi)
# # print(dataset.size().getInfo())
# # print(dataset.aggregate_array('system:id').getInfo())

# # sentinel-2
# # 加载sentinel 图像
# def maskS2clouds(image):
#     qa = image.select('QA60')
#     cloudBitMask = 1 << 10
#     cirrusBitMask = 1 << 11
#     mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
#              .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
#     return image.updateMask(mask).divide(10000)
# sen1 = ee.Image('COPERNICUS/S2_SR/20200526T024551_20200526T024552_T51TWL')
# sen2 = ee.Image('COPERNICUS/S2_SR/20200526T024551_20200526T024552_T51TWM')
# sen3 = ee.Image('COPERNICUS/S2_SR/20200526T024551_20200526T024552_T51TWN')
# sen4 = ee.Image('COPERNICUS/S2_SR/20200526T024551_20200526T024552_T51TXL')
# sen5 = ee.Image('COPERNICUS/S2_SR/20200526T024551_20200526T024552_T51TXM')
# sen6 = ee.Image('COPERNICUS/S2_SR/20200526T024551_20200526T024552_T51TXN')
# # sen2 = ee.Image('COPERNICUS/S2_SR/20201225T030131_20201225T030129_T50RKU')
# # sen3 = ee.Image('COPERNICUS/S2_SR/20201225T030131_20201225T030129_T50RLU')

# sen14 = ee.ImageCollection([sen1,sen2,sen3,sen4,sen5,sen6])
# Map.addLayer(maskS2clouds(sen14.mosaic().clip(roi)),{'min': 0.0,'max': 0.3,'bands': ['B8', 'B4', 'B3']},'sen14')

In [ ]:
# 'LANDSAT/LC08/C01/T1_SR/LC08_119028_20201111' 
# 'LANDSAT/LC08/C01/T1_SR/LC08_120027_20200526' 
# 'LANDSAT/LC08/C01/T1_SR/LC08_120028_20200526'
# image = maskSR(water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_120028_20200526'))).clip(roi)
# visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
# Map.addLayer(image.clip(roi), visParams,'image')


# a = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_122033_20200828').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
# a = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_120028_20200526')
# image = maskSR(water_index(a)).clip(roi)
# visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
# Map.addLayer(image, visParams,'image')

In [ ]:

#  'LANDSAT/LC08/C01/T1_SR/LC08_120028_20150614', 
#  'LANDSAT/LC08/C01/T1_SR/LC08_120028_20150801', 
#  'LANDSAT/LC08/C01/T1_SR/LC08_120028_20151004'

# a = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_120028_20150614')
# image = maskSR(water_index(a))
# visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
# Map.addLayer(image.clip(roi), visParams,'image')

# 直方图

In [ ]:
# 直方图
%matplotlib inline
# import matplotlib.pyplot as plt
from bqplot import pyplot as plt
from bqplot import Bars

index = 'NDWI'
region = 'region A'
histogram = image.select(index).reduceRegion(**{
  'reducer': ee.Reducer.histogram(100),
  'geometry': image.geometry(),
  'scale': 30,
  'bestEffort': True
})

hist_dict = histogram.getInfo()
x = hist_dict[index]['bucketMeans']
y = hist_dict[index]['histogram']

fig = plt.figure(title = region + ' / '+ index)
axes_options = {"x": dict(label = index + ' value')}
line = plt.bar(x, y,axes_options=axes_options)
line.scales["x"].min = -1
line.scales["x"].max = 1
plt.show()

In [ ]:
# 直方图
%matplotlib inline
# import matplotlib.pyplot as plt
from bqplot import pyplot as plt
from bqplot import Bars

index = 'mNDWI'
region = 'region A'
histogram = image.select(index).reduceRegion(**{
  'reducer': ee.Reducer.histogram(100),
  'geometry': image.geometry(),
  'scale': 30,
  'bestEffort': True
})

hist_dict = histogram.getInfo()
x = hist_dict[index]['bucketMeans']
y = hist_dict[index]['histogram']

fig = plt.figure(title = region + ' / '+ index)
axes_options = {"x": dict(label = index + ' value')}
line = plt.bar(x, y,axes_options=axes_options)
line.scales["x"].min = -1
line.scales["x"].max = 1
plt.show()
# fig

# k-mean聚类，RF训练

In [ ]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [ ]:
region = 'daqing'
sample_region = 'DAQING'

In [ ]:
def sumSquare(i,histogram,counts,means,size,mean):
    aCounts = counts.slice(0, 0, i)
    aCountTotal = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCountTotal)
       
    bCounts = counts.slice(0, i, size)
    bCountTotal = bCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    bMeans = means.slice(0, i, size)
    bMean = bMeans.multiply(bCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(bCountTotal)

    return aCountTotal.multiply(aMean.subtract(mean).pow(2)).add(bCountTotal.multiply(bMean.subtract(mean).pow(2)))
# 'B5_histogram'和 polygon、B5 要根据情景进行变更,
# Return the DN that maximizes interclass variance in B5 (in the region).

def otsu(image,band):
    histogram = image.select(band).reduceRegion(**{
        'reducer': ee.Reducer.histogram(100),
        'geometry': image.geometry(), 
        'scale': 30,
        'bestEffort': True
    })
    counts = ee.Array(ee.Dictionary(histogram.get(band)).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram.get(band)).get('bucketMeans'))
    size = means.length().get([0])
    mean = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(counts.reduce(ee.Reducer.sum(), [0]).get([0]))
    
    indices = ee.List.sequence(1,size)
    # Compute between sum of squares, where each mean partitions the data.
    bss = indices.map(lambda i : sumSquare(i,histogram.get(band),counts,means,size,mean))
    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])

threshold = otsu(image,'AWEI')
print(threshold.getInfo())
threshold = otsu(image,'NDWI')
print(threshold.getInfo())
threshold = otsu(image,'mNDWI')
print(threshold.getInfo())

In [ ]:
def otsu(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)

    indices = ee.List.sequence(1, size)

  # Compute between sum of squares, where each mean partitions the data.

    def func_xxx(i):
        aCounts = counts.slice(0, 0, i)
        aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans = means.slice(0, 0, i)
        aMean = aMeans.multiply(aCounts) \
            .reduce(ee.Reducer.sum(), [0]).get([0]) \
            .divide(aCount)
        bCount = total.subtract(aCount)
        bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(
              bCount.multiply(bMean.subtract(mean).pow(2)))

    bss = indices.map(func_xxx)

    # Return the mean value corresponding to the maximum BSS.
    return means.sort(bss).get([-1])

def otsu_value(image):
    histogram = image.select('NDWI').reduceRegion(**{
      'reducer': ee.Reducer.histogram(100),
      'geometry': image.geometry(),
      'scale': 30,
      'bestEffort': True
    })
    threshold = otsu(histogram.get('NDWI'))  
    return image.set({"threshold": threshold})    

In [ ]:
a = otsu_value(image)
print(a.get('threshold').getInfo())

In [ ]:
# ## 方法一
# histogram = image.select('NDWI').reduceRegion(**{
#   'reducer': ee.Reducer.histogram(100),
#   'geometry': image.geometry(),
#   'scale': 30,
#   'bestEffort': True
# })
# def otsu(histogram):
#     counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
#     means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
#     size = means.length().get([0])
#     total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
#     sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
#     mean = sum.divide(total)

#     indices = ee.List.sequence(1, size)

#   # Compute between sum of squares, where each mean partitions the data.

#     def func_xxx(i):
#         aCounts = counts.slice(0, 0, i)
#         aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
#         aMeans = means.slice(0, 0, i)
#         aMean = aMeans.multiply(aCounts) \
#             .reduce(ee.Reducer.sum(), [0]).get([0]) \
#             .divide(aCount)
#         bCount = total.subtract(aCount)
#         bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
#         return aCount.multiply(aMean.subtract(mean).pow(2)).add(
#               bCount.multiply(bMean.subtract(mean).pow(2)))

#     bss = indices.map(func_xxx)

#     # Return the mean value corresponding to the maximum BSS.
#     return means.sort(bss).get([-1])
# threshold = otsu(histogram.get('N'))
# print('threshold', threshold.getInfo())

In [ ]:
-0.2349105369902031
-0.3516368104040866
0.11709049858762309

In [ ]:
awei = image.select('AWEI').gt(-0.2349105369902031)
Map.addLayer(awei.clip(roi).selfMask(),{'palette':['#16d60a']},'awei')

ndwi = image.select('NDWI').gt(-0.3516368104040866)
Map.addLayer(ndwi.clip(roi).selfMask(),{'palette':['#7960ff']},'ndwi')

mndwi = image.select('mNDWI').gt(0.11709049858762309)
Map.addLayer(mndwi.clip(roi).selfMask(),{'palette':['#7960ff']},'mndwi')

In [ ]:
totalSamples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_9920' + '_v2')
print(totalSamples.size().getInfo())
label = 'waterclass'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
trainedClassifier = ee.Classifier.smileRandomForest(150).train(totalSamples,label,bands)
new_result = image.select(bands).classify(trainedClassifier).eq(1).rename('waterclass') 
Map.addLayer(new_result.clip(roi).selfMask(),{'palette':['blue']},"new_result") 

In [ ]:
a = ee.FeatureCollection('users/311605001111/auto_sample_awei0/' + region + '_2000')
totalSamples = k_mean(a)
print(totalSamples.size().getInfo())
label = 'waterclass'
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
trainedClassifier2 = ee.Classifier.smileRandomForest(150).train(totalSamples,label,bands)
old_result = image.select(bands).classify(trainedClassifier2).eq(1).rename('waterclass') 
Map.addLayer(old_result.clip(roi).selfMask(),{'palette':['purple']},"old_result") 

In [ ]:
q = new_result.subtract(mndwi)
Map.addLayer(q.clip(roi),{'palette':['red','white','blue']},"q") 

In [ ]:
awei = image.select('AWEI').gt(0)
Map.addLayer(awei.selfMask(),{'palette':['#16d60a']},'awei')

ndwi = image.select('NDWI').gt(0)
Map.addLayer(ndwi.selfMask(),{'palette':['#7960ff']},'ndwi')

mndwi = image.select('mNDWI').gt(0)
Map.addLayer(mndwi.selfMask(),{'palette':['#7960ff']},'mndwi')

# 制图

In [ ]:
# 120027
# re = [114.5845, 30.3420, 114.6457, 30.3818]
# re = [123.9587, 46.8551, 123.9959, 46.8764]
# re = [124.2623, 46.4508, 124.3289, 46.4910]
# re = [124.6407, 46.7487, 124.6622, 46.7698]
# re = [124.7252, 46.7461,124.7462, 46.7587]
# re = [124.6761, 46.7060,124.6918, 46.7161]
# 120028
# re = [124.0545, 45.8369, 124.1701, 45.9004]
# re = [124.3168, 45.8474, 124.3614, 45.8782]
# 119028
# re = [124.1115, 45.9352, 124.2177, 46.0075]

In [ ]:
region = 'daqing'
sample_region = 'DAQING'

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
re = [123.9355, 45.8362,125.3055, 46.8762]
# re = [124.0277, 46.2409,124.1474, 46.3380]
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
# image = maskSR(water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_119028_20201111'))).clip(roi)
# image = water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_120028_20150614')).clip(roi)

a = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_119028_20000816').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
image = maskSR(water_index(a)).clip(roi)
# image = water_index(a).clip(roi)
refer_img = image.visualize(**visParams)
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(refer_img, region=re)
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'Raw image (20000816)', fontsize=28)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

item = [['new_result','Our result'],['ndwi','NDWI (T=0)'],['mndwi','mNDWI (T=0.118)'],['awei','AWEI (T=-0.235)']]
# re = [123.9355, 45.8362,125.3055, 46.8762]
re = [124.0277, 46.2409,124.1474, 46.3380]
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}
# image = maskSR(water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_120028_20150614'))).clip(roi)
# image = water_index(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_120028_20150614')).clip(roi)

a = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_119028_20000816').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
# image = maskSR(water_index(a)).clip(roi)
image = water_index(a).clip(roi)
for it in item:
    if it[0] == 'new_result':
        totalSamples = ee.FeatureCollection('users/311605001111/SAMPLE_' + sample_region + '/' + region + '_kmean100_9920' + '_v2')
        label = 'waterclass'
        bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
        trainedClassifier = ee.Classifier.smileRandomForest(150).train(totalSamples,label,bands)
        classify_result = image.select(bands).classify(trainedClassifier).eq(1).rename('waterclass')  
    elif it[0] == 'ndwi':
        classify_result = image.select('NDWI').gt(0)
    elif it[0] == 'mndwi':
        classify_result = image.select('mNDWI').gt(0.11709049858762309)
    elif it[0] == 'awei':
        classify_result = image.select('AWEI').gt(-0.2349105369902031)
    refer_img = image.visualize(**visParams)
    pw = classify_result.selfMask().visualize(**{'palette':['blue']})
    blend = refer_img.blend(pw)
    fig = plt.figure(figsize=(12, 8))
    ax = cartoee.get_map(blend, region=re)
    cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
    ax.set_title(label = it[1], fontsize=28)